In [2]:
%pylab inline
import matplotlib
import numpy as np

Populating the interactive namespace from numpy and matplotlib


This is an example of how to set up notebooks for use in Sphinx docs. You can choose to have some cells hidden; they execute in the notebook but are not shown in the HTML output. This is useful for imports and setup. 

To do this, in Jupyter go to the View menu, select the Cell Toolbar submenu, and select "Edit Metadata". This will make each cell have a new "Edit Metadata" button. The metadata is a dict for each cell. Add ` "nbsphinx": "hidden"` to a given cell and it will be hidden. This has been done for the above cell and for this one. 

# MIRAGE Quick Start

**Important:** Before proceeding, ensure you have set the MIRAGE_DATA environment variable to point to the top level directory that contains the reference files associated with MIRAGE.

In [3]:
from glob import glob
from mirage import imaging_simulator
from mirage.apt import apt_inputs
from mirage.yaml import yaml_generator

### Create `mirage` input yaml files from an APT file

**NOTE**: To see an example observation list file, see the [Simulating Observations from an APT File](https://jwst-mirage.readthedocs.io/en/stable/yaml_generator.html) page.

In [ ]:
apt_xml_file = 'my_apt_file.xml'
apt_pointing_file = 'my_apt_file.pointing'
observation_list_file = 'observation_list.yaml'

In [ ]:
# Create a series of data simulator input yaml files
# from APT files
yam = yaml_generator.SimInput()

# Point to the xml and pointing files from your APT proposal
yam.input_xml = apt_xml_file
yam.pointing_file = apt_pointing_file

# Output directory for the collection of yaml files
yam.output_dir = './yaml_files'

# Output directory for the simulated observations
yam.simdata_output_dir = './imaging_data/'

# Observation table that lists the source catalogs to use with each filter/observation
yam.observation_table = observation_list_file

# Output data type. "raw", "linear", or "linear,raw" for both
yam.datatype = 'raw'

# Create the yaml files
yam.reffile_setup()
yam.create_inputs()

### Run `mirage` on all yaml files

In [ ]:
# Create a list of all yaml files to be run
yaml_files = glob('./yaml_files/*yaml')

#### run in series:

In [ ]:
for yaml_file in yaml_files:
    sim = imaging_simulator.ImgSim()
    sim.paramfile = yaml_file
    sim.create()

#### run in parallel:

In [ ]:
# Small wrapper function for mirage so that we can parallelize it
def run_mirage_imaging(paramfile):
    img = imaging_simulator.ImgSim()
    img.paramfile = paramfile
    img.create()

In [ ]:
from multiprocessing import Pool

n_procs = 5 # number of cores to use
with Pool(n_procs) as pool:
    pool.map(run_mirage_imaging, yaml_files)